<a href="https://colab.research.google.com/github/wanadzhar913/bank-transaction-classification/blob/master/notebooks/02_model_building_deberta_text_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install datasets huggingface_hub -q

In [ ]:
# import transformers
# import datasets
# import huggingface_hub
# import sklearn
# import torch
# import numpy as np
# import pandas as pd

# print(f'transformers version: {transformers.__version__}')
# print(f'datasets version: {datasets.__version__}')
# print(f'huggingface_hub version: {huggingface_hub.__version__}')
# print(f'scikit-learn version: {sklearn.__version__}')
# print(f'torch version: {torch.__version__}')
# print(f'numpy version: {np.__version__}')
# print(f'pandas version: {pd.__version__}')

transformers version: 4.47.1
datasets version: 3.2.0
huggingface_hub version: 0.27.1
scikit-learn version: 1.6.0
torch version: 2.5.1+cu121
numpy version: 1.26.4
pandas version: 2.2.2


In [ ]:
import os
import json
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datasets import load_dataset, Dataset
from huggingface_hub import create_repo, notebook_login
from sklearn.metrics import classification_report, average_precision_score, \
                            accuracy_score, f1_score, precision_score, recall_score

import torch
from transformers import AutoTokenizer, AutoConfig, pipeline, \
                         DebertaV2ForSequenceClassification

pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [ ]:
notebook_login()

##### 1.0 Load dataset

In [ ]:
!mkdir data/
!cd data/ && wget https://raw.githubusercontent.com/wanadzhar913/bank-transaction-classification/refs/heads/master/data/train.csv -q
!cd data/ && wget https://raw.githubusercontent.com/wanadzhar913/bank-transaction-classification/refs/heads/master/data/test.csv -q

In [ ]:
train = pd.read_csv('data/train.csv')
train.head(3)

,client_id,bank_id,account_id,txn_id,txn_date,description,amount,description_stem,description_clean_len,day_monday,...,monthly_transaction_count_atm,monthly_transaction_count_bank_fee,monthly_transaction_count_payment,IS_INTERESTED_INVESTMENT,IS_INTERESTED_BUILD_CREDIT,IS_INTERESTED_INCREASE_INCOME,IS_INTERESTED_PAY_OFF_DEBT,IS_INTERESTED_MANAGE_SPENDING,IS_INTERESTED_GROW_SAVINGS,category
0,577,1,2,90,2023-08-14 00:00:00,Online Banking transfer from CHK 6479 Confirmation# 1425,6.0,onlin bank transfer chk confirm,5,1,...,0,0,0,0,0,1,0,1,0,Internal Account Transfer
1,94,8,8,159,2023-09-18 00:00:00,DEPOSIT,5.0,deposit,1,1,...,0,0,0,0,0,0,0,0,0,Payroll
2,880,770,887,99720,2023-06-16 00:00:00,Bulk 7202 Apple 1920 EAST WISCONSI APPLETON WIUS,-10.9,bulk appl east wisconsi appleton wiu,6,0,...,1891,12,14,0,0,0,0,0,0,Uncategorized


In [ ]:
test = pd.read_csv('data/test.csv')
test.head(3)

,client_id,bank_id,account_id,txn_id,txn_date,description,amount,description_stem,description_clean_len,day_monday,...,monthly_transaction_count_atm,monthly_transaction_count_bank_fee,monthly_transaction_count_payment,IS_INTERESTED_INVESTMENT,IS_INTERESTED_BUILD_CREDIT,IS_INTERESTED_INCREASE_INCOME,IS_INTERESTED_PAY_OFF_DEBT,IS_INTERESTED_MANAGE_SPENDING,IS_INTERESTED_GROW_SAVINGS,category
0,880,828,953,24000,2023-08-11 00:00:00,SECU Foundation,-0.200,secu foundat,2,0,...,1891,12,14,0,0,0,0,0,0,Restaurants
1,880,17,17,178308,2023-09-26 00:00:00,SELF LENDER AUSTIN TX 23267 Debit Card Purchase 09/23 10:20a #6410,-5.240,self lender austin tx debit card purchas,7,0,...,1891,12,14,0,0,0,0,0,0,Uncategorized
2,880,489,558,89624,2023-06-05 00:00:00,RECURRING PAYMENT AUTHORIZED ON 06/02 GEICO *AUTO 1036 DC S583153489705993 111,-50.398,recur payment author geico auto dc,6,1,...,1891,12,14,0,0,0,0,0,0,Insurance


In [ ]:
# to ensure the presence of every class in each dataset split
assert train.category.nunique() == test.category.nunique(), 'Num. of categories are different'

category = list(train.category.unique())

# Sorting the categories alphabetically
sorted_categories = sorted(category)

# Generating the category mapping dictionary with values starting from 0
category_dict = {category: index for index, category in enumerate(sorted_categories)}
category_dict

{'ATM': 0,
 'Arts and Entertainment': 1,
 'Bank Fee': 2,
 'Bank Fees': 3,
 'Check Deposit': 4,
 'Clothing and Accessories': 5,
 'Convenience Stores': 6,
 'Department Stores': 7,
 'Digital Entertainment': 8,
 'Food and Beverage Services': 9,
 'Gas Stations': 10,
 'Gyms and Fitness Centers': 11,
 'Healthcare': 12,
 'Insurance': 13,
 'Interest': 14,
 'Internal Account Transfer': 15,
 'Loans': 16,
 'Payment': 17,
 'Payroll': 18,
 'Restaurants': 19,
 'Service': 20,
 'Shops': 21,
 'Supermarkets and Groceries': 22,
 'Tax Refund': 23,
 'Telecommunication Services': 24,
 'Third Party': 25,
 'Transfer': 26,
 'Transfer Credit': 27,
 'Transfer Debit': 28,
 'Transfer Deposit': 29,
 'Travel': 30,
 'Uncategorized': 31,
 'Utilities': 32}

In [ ]:
# Define a function to map our categories to unique IDs
def map_category(example):
    example['category'] = category_dict.get(example['category'], -1)  # -1 for unknown categories
    return example

In [ ]:
df_train = Dataset.from_pandas(train)
df_train

Dataset({
    features: ['client_id', 'bank_id', 'account_id', 'txn_id', 'txn_date', 'description', 'amount', 'description_stem', 'description_clean_len', 'day_monday', 'day_friday', 'day_weekend', 'monthly_transaction_count', 'monthly_transaction_count_loans', 'monthly_transaction_count_transfer_credit', 'monthly_transaction_count_transfer_deposit', 'monthly_transaction_count_payroll', 'monthly_transaction_count_uncategorized', 'monthly_transaction_count_restaurants', 'monthly_transaction_count_check_deposit', 'monthly_transaction_count_third_party', 'monthly_transaction_count_food_and_beverage_services', 'monthly_transaction_count_internal_account_transfer', 'monthly_transaction_count_shops', 'monthly_transaction_count_supermarkets_and_groceries', 'monthly_transaction_count_telecommunication_services', 'monthly_transaction_count_bank_fees', 'monthly_transaction_count_utilities', 'monthly_transaction_count_insurance', 'monthly_transaction_count_digital_entertainment', 'monthly_transac

In [ ]:
df_test = Dataset.from_pandas(test)
df_test

Dataset({
    features: ['client_id', 'bank_id', 'account_id', 'txn_id', 'txn_date', 'description', 'amount', 'description_stem', 'description_clean_len', 'day_monday', 'day_friday', 'day_weekend', 'monthly_transaction_count', 'monthly_transaction_count_loans', 'monthly_transaction_count_transfer_credit', 'monthly_transaction_count_transfer_deposit', 'monthly_transaction_count_payroll', 'monthly_transaction_count_uncategorized', 'monthly_transaction_count_restaurants', 'monthly_transaction_count_check_deposit', 'monthly_transaction_count_third_party', 'monthly_transaction_count_food_and_beverage_services', 'monthly_transaction_count_internal_account_transfer', 'monthly_transaction_count_shops', 'monthly_transaction_count_supermarkets_and_groceries', 'monthly_transaction_count_telecommunication_services', 'monthly_transaction_count_bank_fees', 'monthly_transaction_count_utilities', 'monthly_transaction_count_insurance', 'monthly_transaction_count_digital_entertainment', 'monthly_transac

For this experiment, **we'll only take text as our input**. Hence, we're only interested the *description* column for our input features, X.

In [ ]:
train_X = df_train.select_columns(["description"])
test_X = df_test.select_columns(["description"])

train_Y = df_train.select_columns(["category"]).map(map_category)
test_Y = df_test.select_columns(["category"]).map(map_category)

Map:   0%|          | 0/199260 [00:00<?, ? examples/s]

Map:   0%|          | 0/49816 [00:00<?, ? examples/s]

In [ ]:
print(train_X[20])
print(train_Y[20])

{'description': 'Transfer From -Graham Dorsa 0323'}
{'category': 27}


##### 2.0 Load & set model configs

In [ ]:
config = AutoConfig.from_pretrained('microsoft/deberta-v3-base')
config.num_labels = len(set(category))
config.vocab = category

In [ ]:
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', config = config)
model.cuda()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
!nvidia-smi

Sun Jan 12 00:44:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              49W / 400W |   1181MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

##### 3.0 Train/finetune model

In [ ]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 1e-5, eps=1e-08, betas=(0.9,0.999))

# Add a ReduceLROnPlateau scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(trainer, mode='max', patience=2, factor=0.5, verbose=True)

In [ ]:
batch_size = 64
epoch = 15 # due to compute (and time) constraints, we set a maximum of 15 epochs

best_dev_acc = -np.inf
patient = 3
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    training_losses = []
    grad_norms = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y['category'][i: i + batch_size])

        padded = tokenizer(x['description'], padding = 'longest', return_tensors = 'pt')

        padded['labels'] = torch.from_numpy(y).long()
        for k in padded.keys():
            padded[k] = padded[k].cuda()

        padded.pop('token_type_ids', None)

        loss, pred = model(**padded, return_dict = False)
        loss.backward()

        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 5.0)
        trainer.step()
        training_losses.append(float(loss))
        grad_norms.append(float(grad_norm))

    model.eval()
    dev_predicted = []
    validation_losses = []
    for i in range(0, len(test_X), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y['category'][i: i + batch_size])

        padded = tokenizer(x['description'], padding = 'longest', return_tensors = 'pt')
        padded['labels'] = torch.from_numpy(y).long()
        for k in padded.keys():
            padded[k] = padded[k].cuda()

        with torch.no_grad():
            loss, pred = model(**padded, return_dict = False)
        dev_predicted.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())
        validation_losses.append(float(loss))

    # some notes on the warnings below: https://stackoverflow.com/questions/43162506/undefinedmetricwarning-f-score-is-ill-defined-and-being-set-to-0-0-in-labels-wi
    accuracy = accuracy_score(dev_predicted, test_Y['category'])
    f1 = f1_score(dev_predicted, test_Y['category'], average='weighted')
    precision = precision_score(dev_predicted, test_Y['category'], average='weighted')
    recall = recall_score(dev_predicted, test_Y['category'], average='weighted')

    print(f'epoch: {e}, lr: {scheduler.get_last_lr()[0]:.8f}, grad_norm: {np.mean(grad_norms)}, training_loss: {np.mean(training_losses):.4f}, validation_loss: {np.mean(validation_losses)}, accuracy: {accuracy}, f1: {f1}, precision: {precision}, recall: {recall}')

    # Call scheduler.step() with the f1 score
    scheduler.step(f1)

    if f1 >= best_dev_acc:
        best_dev_acc = f1
        current_patient = 0
        model.save_pretrained('debertav3-finetune-banking-transaction-classification')
    else:
        current_patient += 1

    if current_patient >= patient:
        break

    if e == epoch - 1:
        print(f'Final dev accuracy: {best_dev_acc}')
        model.save_pretrained('debertav3-finetune-banking-transaction-classification')

100%|██████████| 3114/3114 [13:35<00:00,  3.82it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch: 0, lr: 0.00001000, grad_norm: 5.9528155882311085, training_loss: 0.6011, validation_loss: 0.3389442953279909, accuracy: 0.8848964188212622, f1: 0.892675706042445, precision: 0.9089973118437291, recall: 0.8848964188212622


100%|██████████| 3114/3114 [13:30<00:00,  3.84it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch: 1, lr: 0.00001000, grad_norm: 4.864737870873896, training_loss: 0.2675, validation_loss: 0.2787521743575477, accuracy: 0.8992291633210213, f1: 0.9032402981228885, precision: 0.9122518630841268, recall: 0.8992291633210213


100%|██████████| 3114/3114 [13:26<00:00,  3.86it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

100%|██████████| 3114/3114 [13:25<00:00,  3.87it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch: 11, lr: 0.00000500, grad_norm: 3.154900153085599, training_loss: 0.0869, validation_loss: 0.352199846919296, accuracy: 0.9100088325036133, f1: 0.9113069189221518, precision: 0.9139972063646302, recall: 0.9100088325036133


100%|██████████| 3114/3114 [13:28<00:00,  3.85it/s]


epoch: 12, lr: 0.00000500, grad_norm: 3.349912312343554, training_loss: 0.0843, validation_loss: 0.3534109901005596, accuracy: 0.9116548899951823, f1: 0.91236915533028, precision: 0.9142650070401553, recall: 0.9116548899951823


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
label2id = {category: index for index, category in enumerate(sorted_categories)}
id2label = {index: category for index, category in enumerate(sorted_categories)}

In [ ]:
model.config.label2id = label2id
model.config.id2label = id2label

**NOTE:** *I made an error in the below! The `config.vocab` variable isn't sorted. Hence, the confusion matrix below has the wrong columns assigned to it. It's actually `Tax Refund` that has 0 elements classified correctly.*

In [ ]:
print(
    classification_report(
        dev_predicted, test_Y['category'], target_names = config.vocab,
        digits = 5
    )
)

                            precision    recall  f1-score   support

 Internal Account Transfer    1.00000   0.99561   0.99780      1139
                   Payroll    0.91026   0.94667   0.92810        75
             Uncategorized    0.85714   1.00000   0.92308         6
            Transfer Debit    0.98829   0.99764   0.99294      1269
                 Bank Fees    0.83333   0.89744   0.86420        39
               Third Party    0.83087   0.85908   0.84474       589
                     Loans    0.74942   0.68666   0.71667      3785
  Clothing and Accessories    0.66319   0.75821   0.70752       335
                 Insurance    0.95491   0.92809   0.94131       890
                     Shops    0.79412   0.69231   0.73973        78
        Convenience Stores    0.49848   0.57913   0.53579      1984
               Restaurants    1.00000   0.82353   0.90323        17
           Transfer Credit    0.59459   0.91667   0.72131        24
              Gas Stations    0.93884   0.92470

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# a quick look at the "Arts and Entertainment" column
test.query('category == "Arts and Entertainment"').description

,description
794,bet365.OH 1036 NJ
1977,bet365.OH 1036 NJ
2103,bet365.OH 1036 NJ 08/01
3074,MONEY TRANSFER AUTHORIZED ON 09/28 FROM (SPORTSBOOK) DRAFTKINGS MA S463271477360776 111
4936,MONEY TRANSFER AUTHORIZED ON 09/20 FROM (SPORTSBOOK) DRAFTKINGS MA S463263432342860 111
...,...
47643,bet365.OH 1036 NJ 09/16
48101,PURCHASE AUTHORIZED ON 09/19 bet365.CO 1036 NJ S383263192924746 111
48897,PURCHASE AUTHORIZED ON 09/09 bet365.CO 1036 NJ S583252648236901 111
48994,MONEY TRANSFER AUTHORIZED ON 09/08 FROM (SPORTSBOOK) DRAFTKINGS MA S303251831353056 111


### 4.0 Save and upload to HuggingFace

In [ ]:
create_repo("wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only", repo_type="model")

RepoUrl('https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only', endpoint='https://huggingface.co', repo_type='model', repo_id='wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only')

In [ ]:
model.push_to_hub('wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only', safe_serialization = True)

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only/commit/61ca79ccc93087fe62d3867a3a9a7b6d6777e42e', commit_message='Upload DebertaV2ForSequenceClassification', commit_description='', oid='61ca79ccc93087fe62d3867a3a9a7b6d6777e42e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only', endpoint='https://huggingface.co', repo_type='model', repo_id='wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub('wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only', safe_serialization = True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only/commit/993597035052e75abbd3fea257f76245bfc7d89a', commit_message='Upload tokenizer', commit_description='', oid='993597035052e75abbd3fea257f76245bfc7d89a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only', endpoint='https://huggingface.co', repo_type='model', repo_id='wanadzhar913/debertav3-finetuned-banking-transaction-classification-text-only'), pr_revision=None, pr_num=None)